In [83]:
## Imports
import pandas as pd
import numpy as np
import random

In [84]:
def create_batch():
    ## load 200 anchors images
    df_anchor = pd.read_csv("ISIC-2017_anchor_images.csv")
    #print(df_anchor.head())

    ## load 1000 (2* X, where X = 500) "unlabeled" images
    df_unlabeled = pd.read_csv("ISIC_2020_unlabeled_images.csv")
    #print(df_unlabeled.head())
    
    ## Create skeleton for the batch-file
    df_batch = pd.DataFrame()
    df_temp_anchor = pd.DataFrame()
    df_temp_unlabeled_big = pd.DataFrame()
    df_temp_unlabeled_A = pd.DataFrame()
    df_temp_unlabeled_B = pd.DataFrame()

    ## Create triplets randomly in the range X (now 10)
    #
    ## First we sample X-amount of random pictures, and save their ID and their labels
    ## While also adding the url:
    # 

    X = 500
    #Find X random anchor pictures
    #Now Anchor list is 200 and we want it to be 1000
    df_anchor = pd.concat([df_anchor]*5)
    df_temp_anchor[["image1_title", "image1_label"]] = df_anchor[["image_id", "cancer"]].sample(frac=1)
    print("temp_anchor shuffled!")
    df_temp_anchor[["image1_url"]] = "https://github.com/vcheplygina/crowdsim/blob/main/Extra/Data/batch/" + df_temp_anchor[["image1_title"]].values + "_resized.jpg?raw=true"

    #Find X random and different pictures (now we can hope each unlabeled picture is listed 4 times)
    #Now unlabeled has 500 pictures, we want it to be 2000
    df_unlabeled = pd.concat([df_unlabeled]*4)
    df_temp_unlabeled_big = df_unlabeled[["image_id", "cancer"]].sample(frac=1)
    print("temp_unlabeled shuffled!")

    # Split into 2 groups of X
    df_temp_unlabeled_A[["image2_title", "image2_label"]] = df_temp_unlabeled_big[0:X*2]
    df_temp_unlabeled_A[["image2_url"]] = "https://github.com/vcheplygina/crowdsim/blob/main/Extra/Data/batch/" + df_temp_unlabeled_A[["image2_title"]].values + "_resized.jpg?raw=true"

    df_temp_unlabeled_B[["image3_title", "image3_label"]] = df_temp_unlabeled_big[X*2:X*4]
    df_temp_unlabeled_B[["image3_url"]] = "https://github.com/vcheplygina/crowdsim/blob/main/Extra/Data/batch/" + df_temp_unlabeled_B[["image3_title"]].values + "_resized.jpg?raw=true"

    #
    ## Then we reset their indices, and join them "horisontally"
    #

    df_temp_anchor = df_temp_anchor.reset_index(drop=True)
    df_temp_unlabeled_A = df_temp_unlabeled_A.reset_index(drop=True)
    df_temp_unlabeled_B = df_temp_unlabeled_B.reset_index(drop=True)

    #
    ## Joining here into 1 dataframe
    #
    df_batch = df_temp_anchor.join(df_temp_unlabeled_A).join(df_temp_unlabeled_B)
    print(df_batch.tail())

In [88]:
# If two "A" and "B" are the same redo:
batch_succesful = False
while not batch_succesful:
    batch_succesful = True
    create_batch()
    for i in range(100):
        if df_batch.iloc[i]['image2_title'] == df_batch.iloc[i]["image3_title"]:
            batch_succesful = False
            print ("this is bad! Trying again... count:", count)
            
print("batch created succesfully!")

temp_anchor shuffled!
temp_unlabeled shuffled!
     image1_title  image1_label  \
995  ISIC_0004168           0.0   
996  ISIC_0013500           1.0   
997  ISIC_0000003           0.0   
998  ISIC_0013832           1.0   
999  ISIC_0009882           1.0   

                                            image1_url  image2_title  \
995  https://github.com/vcheplygina/crowdsim/blob/m...  ISIC_8229307   
996  https://github.com/vcheplygina/crowdsim/blob/m...  ISIC_2273490   
997  https://github.com/vcheplygina/crowdsim/blob/m...  ISIC_6687278   
998  https://github.com/vcheplygina/crowdsim/blob/m...  ISIC_7991932   
999  https://github.com/vcheplygina/crowdsim/blob/m...  ISIC_7236143   

     image2_label                                         image2_url  \
995             0  https://github.com/vcheplygina/crowdsim/blob/m...   
996             0  https://github.com/vcheplygina/crowdsim/blob/m...   
997             0  https://github.com/vcheplygina/crowdsim/blob/m...   
998             0  ht

In [89]:
## Create the csv file:
df_batch.to_csv("batch_for_crowdsim.csv")